In [1]:
%run -m ipy_startup
%run -m ipy_logging false
%run -m ipy_seaborn
%matplotlib inline

In [2]:
import random
from deap import algorithms, base, creator, tools

In [40]:
from sklearn import model_selection

In [55]:
x = np.zeros((5, 5))
y = np.ones((5, 1))
if len(y.shape) == 1:
    y = np.reshape(np.ones((5)), (-1,1))
np.concatenate((x, y), axis=1)

array([[ 0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  1.]])

In [42]:
from sklearn.model_selection import _BaseKFold

class StratifiedMultiKFold(_BaseKFold):

    def __init__(self, n_splits=3, shuffle=False, random_state=None):
        super(StratifiedMultiKFold, self).__init__(n_splits, shuffle, random_state)
        
#     def _iter_test_indices(self, X, y=None, groups=None):
#         n_samples = _num_samples(X)
#         indices = np.arange(n_samples)
#         if self.shuffle:
#             check_random_state(self.random_state).shuffle(indices)

#         n_splits = self.n_splits
#         fold_sizes = (n_samples // n_splits) * np.ones(n_splits, dtype=np.int)
#         fold_sizes[:n_samples % n_splits] += 1
#         current = 0
#         for fold_size in fold_sizes:
#             start, stop = current, current + fold_size
#             yield indices[start:stop]
#             current = stop

    def _iter_test_indices(self, X, y=None, groups=None):
        if len(y.shape) == 1:
            y = np.reshape(y, (-1,1))
        if len(X.shape) == 1:
            X = np.reshape(X, (-1,1))
            
        D = X
        if y is not None:
            D = np.concatenate((X, y), axis=1)
            
        values = []
        for i in range(D.shape[1]):
            values.append(np.unique())
        
        n_samples = _num_samples(X)
        indices = np.arange(n_samples)
        if self.shuffle:
            check_random_state(self.random_state).shuffle(indices)

        n_splits = self.n_splits
        fold_sizes = (n_samples // n_splits) * np.ones(n_splits, dtype=np.int)
        fold_sizes[:n_samples % n_splits] += 1
        current = 0
        for fold_size in fold_sizes:
            start, stop = current, current + fold_size
            yield indices[start:stop]
            current = stop

    def split(self, X, y, groups=None):
        y = check_array(y, ensure_2d=False, dtype=None)
        return super(StratifiedMultiKFold, self).split(X, y, groups)

IndentationError: unindent does not match any outer indentation level (<ipython-input-42-4d1062a36c5d>, line 55)

In [ ]:
def get_cv_index(d):
    

In [3]:
# x = [1,1,2,2,3,4]
# lvls = [1,2,3,4,5]

def get_proportion_present(x, lvls):
    lvl, lvl_ct = np.unique(x, return_counts=True)
    lvl_unexpected = np.setdiff1d(lvl, lvls)
    if len(lvl_unexpected) > 0:
        raise ValueError(
            'The following elements within an individual had values that were not expected.  '\
            'Expected values = {}, Unique Values Found = {}, Unexpected Values = {}'
            .format(lvls, lvl, lvl_unexpected)
        )
    lvl_missing = np.setdiff1d(lvls, lvl)
    lvl_p = np.concatenate((lvl_ct / len(x), np.repeat(0., len(lvl_missing))))
    return lvl_p

In [13]:
k = 3

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
#toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("attr_int", np.random.randint, 0, k)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, n=50)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evaluateIndividual(individual):
    p = get_proportion_present(individual, np.arange(k))
    v1 = np.product(p)
    #print(v1)
    return (v1,)

toolbox.register("evaluate", evaluateIndividual)
#toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=k-1, indpb=0.05)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("select", tools.selTournament, tournsize=3)


In [14]:
pop = toolbox.population(n=300)

In [20]:
#result = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=100, verbose=False)

stats = tools.Statistics(key=lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

hof = tools.HallOfFame(10)
pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=10, stats=stats, verbose=True, halloffame=hof)
# print('Current best fitness:', evalOneMax(tools.selBest(pop, k=1)[0]))

gen	nevals	avg      	std        	min    	max     
0  	0     	0.0366255	0.000645738	0.03128	0.036992
1  	191   	0.0366339	0.000568539	0.033264	0.036992
2  	178   	0.036623 	0.000625642	0.032912	0.036992
3  	193   	0.0365961	0.000594833	0.033792	0.036992
4  	173   	0.0366911	0.000503019	0.034272	0.036992
5  	174   	0.0367316	0.000499719	0.03344 	0.036992
6  	162   	0.0367737	0.000419774	0.034496	0.036992
7  	173   	0.0367429	0.000443068	0.034496	0.036992
8  	159   	0.0367588	0.000494471	0.03312 	0.036992
9  	174   	0.0368144	0.000361632	0.034496	0.036992
10 	183   	0.0368001	0.000386871	0.034272	0.036992


In [21]:
logbook.select('avg', 'gen')

([0.036625493333333342,
  0.036633866666666674,
  0.036623040000000009,
  0.036596053333333337,
  0.036691146666666681,
  0.036731626666666677,
  0.036773653333333343,
  0.036742933333333345,
  0.036758773333333342,
  0.036814400000000018,
  0.036800106666666679],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [34]:
x = hof[0]
x.fitness.values

(0.036992000000000011,)

In [29]:
for k, v in hof.:
    print(k , v)

ValueError: too many values to unpack (expected 2)

In [ ]:
pop.